# One-SVM
Принцип метода One-SVM заключается в том, чтобы найти гиперплоскость, которая разделяет нормальные данные от аномалий. Гиперплоскость строится таким образом, чтобы минимизировать ошибки на обучающих данных и максимизировать расстояние между гиперплоскостью и ближайшими точками данных.

Основные шаги метода One-SVM:

Обучение модели на нормальных данных. В этом случае аномалии не используются при обучении модели. Поиск аномалий в тестовых данных. Если точка данных находится далеко от гиперплоскости, то она считается аномалией.

Например, если у нас есть временной ряд, то мы можем использовать One-SVM для поиска аномалий в этом ряде. Мы можем обучить модель на нормальных данных (к примеру, на первых 80% временного ряда) и затем использовать ее для поиска аномалий в тестовых данных (оставшиеся 20% временного ряда). Если точка данных находится далеко от гиперплоскости, то она считается аномалией.

Преимущества метода One-SVM:

Метод не требует большого количества размеченных данных, так как использует только нормальные данные.
Метод хорошо работает с многомерными данными.
Метод может обнаруживать аномалии, которые находятся далеко от нормальных данных.
Недостатки метода One-SVM:

Метод может неправильно классифицировать аномалии, которые находятся близко к нормальным данным.
Метод может быть чувствителен к выбросам в нормальных данных.

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')
# ! cp -r /content/drive/MyDrive/Study/MIPT_magistery/qualification_work/data .

In [1]:
import os
import random
import sys
from pprint import pprint

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler

# ml methods
from sklearn.model_selection import GridSearchCV
from sklearn.svm import OneClassSVM

# metrics
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

def seed_everything(seed):
    # фискирует максимум сидов для корректности сравнения разных экспериментов
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
SEED = 42
seed_everything(SEED)

# Datasets

In [2]:
# Синтетически сгенерированные нормально распределенные данные
def make_norm_data(rows, columns, noise_percent=0.05):
    data_norm = pd.DataFrame(data=np.random.normal(
                            loc=0, scale=1, size=(rows, columns)))
    noise = pd.DataFrame(data=np.random.uniform(
                            low=-6, high=6, size=(int(data_norm.shape[0]*noise_percent), data_norm.shape[1])))

    data_noise = pd.DataFrame()
    for feature in noise.columns:
        filter = (
            (noise[feature] < data_norm[feature].min())
            | (noise[feature] > data_norm[feature].max())
            )
        data_noise = pd.concat([data_noise, noise[filter]])
    data_noise = data_noise.drop_duplicates()
    data_norm['anomaly'] = 0
    data_noise['anomaly'] = 1

    df_norm = pd.concat((data_norm, data_noise))
    return df_norm

df_norm = make_norm_data(1000, 8)
df_norm

,0,1,2,3,4,5,6,7,anomaly
0,0.496714,-0.138264,0.647689,1.523030,-0.234153,-0.234137,1.579213,0.767435,0
1,-0.469474,0.542560,-0.463418,-0.465730,0.241962,-1.913280,-1.724918,-0.562288,0
2,-1.012831,0.314247,-0.908024,-1.412304,1.465649,-0.225776,0.067528,-1.424748,0
3,-0.544383,0.110923,-1.150994,0.375698,-0.600639,-0.291694,-0.601707,1.852278,0
4,-0.013497,-1.057711,0.822545,-1.220844,0.208864,-1.959670,-1.328186,0.196861,0
...,...,...,...,...,...,...,...,...,...
32,0.063984,1.772509,0.031678,-0.098264,-3.938531,-2.655435,-4.093768,-3.652004,1
38,-2.800126,2.593122,-1.268901,2.124984,-0.288220,5.925277,2.746826,5.429896,1
5,2.151800,-0.353991,-1.345218,-1.488481,1.680586,-0.062468,4.189587,4.496424,1
37,-2.554173,0.008819,-0.528646,-1.866863,2.324896,3.114742,4.235808,5.674279,1


In [3]:
# SKAB data

all_files=[]
for root, dirs, files in os.walk("data/skab/"):
    for file in files:
        if file.endswith(".csv"):
             all_files.append(os.path.join(root, file))

# формируем датафрейм
dfs=[]
for path in all_files:
    df = pd.read_csv(path,index_col='datetime',sep=';',parse_dates=True)
    # print(path, df.shape)
    dfs.append(df)
# print('Features:')
# for col in dfs[2].columns:
#     print('\t',col)
dfs = [df for df in dfs if df.shape[1] == 10]
df_skab = pd.concat(dfs)
# print(df_skab.shape)
df_skab = df_skab.drop_duplicates()
df_skab = df_skab.drop('changepoint', axis=1).sort_index()
display(df_skab)

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS,anomaly
datetime,,,,,,,,,
2020-03-09 10:54:34,0.026990,0.040407,1.015510,0.382638,72.9570,25.6198,231.276,31.00100,0.0
2020-03-09 10:54:35,0.026836,0.039138,1.020390,0.054711,72.8664,25.6185,226.683,31.99900,0.0
2020-03-09 10:54:36,0.026597,0.040147,0.733759,0.054711,72.8498,25.6157,231.091,31.00100,0.0
2020-03-09 10:54:37,0.027161,0.040001,1.073280,0.382638,72.7915,25.6115,236.156,31.99900,0.0
2020-03-09 10:54:38,0.026703,0.039407,1.106890,0.054711,72.9818,25.6171,207.464,31.00100,0.0
...,...,...,...,...,...,...,...,...,...
2020-02-08 19:32:15,0.105811,0.193051,1.291030,0.054711,86.1452,33.2491,208.855,122.47500,0.0
2020-02-08 19:32:16,0.060570,0.077031,0.824930,0.382638,86.2657,33.2489,227.430,98.54010,0.0
2020-02-08 19:32:17,0.024764,0.037350,0.293163,0.382638,86.5109,33.2445,228.719,69.18940,0.0


In [4]:
datasets = {
    'df_norm': df_norm,
    'df_skab': df_skab
}

# Метрики

In [5]:
# f1_score
def score_metrics(real_outliers, pred_outliers):
    scores = {}
    # scores['f1_score'] = f1_score(real_outliers, pred_outliers)
    scores = classification_report(real_outliers, pred_outliers)
    return scores

# ML методы

In [6]:
def preprocessing(df):
    X = df.copy()
    y = X.pop('anomaly')

    # preprocessing
    columns = list(X.columns)
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    X = pd.DataFrame(data=X, columns=columns)
    return X, y

def ml_method(X, y, model, params):
    grid = GridSearchCV(
                estimator=model(),
                param_grid=params,
                scoring='f1_micro',
                return_train_score=True).fit(X, y)
    pred = grid.best_estimator_.predict(X)
    pred[pred == 1] = 0
    pred[pred == -1] = 1
    print(grid.best_estimator_)
    print(score_metrics(y, pred))

In [7]:
ml_models = {}
ml_models['OneClassSVM'] = {
    'model': OneClassSVM,
    'params': {
        'nu': [0.001, 0.01, 0.1, 0.5, 1],
    }
}

In [ ]:
for df_name, df in datasets.items():
    print(df_name)
    X, y = preprocessing(df)
    for model in ml_models:
        ml_method(X, y, ml_models[model]['model'], 
                        ml_models[model]['params'])
    print('-' * 10)

df_norm
OneClassSVM(nu=0.001)
              precision    recall  f1-score   support

           0       0.98      0.97      0.97      1000
           1       0.49      0.54      0.51        50

    accuracy                           0.95      1050
   macro avg       0.73      0.76      0.74      1050
weighted avg       0.95      0.95      0.95      1050

----------
df_skab
